In [1]:
import sys
import os
import pandas as pd 
import numpy as np
import os 
import sys 
from code.data_processing import get_all_datasets
import matplotlib.pyplot as plt

from ResultHandling import ResultHandling
print(get_all_datasets())

['cleveland', 'ionosphere', 'ecoli', 'iris', 'mammo_graphic', 'wisconsin_breast_cancer', 'australia', 'postop', 'yeast', 'spec']


# Standard method for composing result handlilngs

In [3]:
def load_params(directory):
    file = open(directory) #'results_finished/bagboost/1/4083559_1_info.txt')
    line = [l for l in file][0].split('{')
    line[-1] = line[-1].replace('}', '')
    line = line[1].split(',')

    f = {}
    for x in line:
        x = x.split(':')
        key = x[0].replace("'",'').strip()
        f[key] = str(x[1]).strip()
        
    return f


In [7]:
from experiments.get_experiment import get_experiment
exp = get_experiment('bagmogp')
print(exp['models'])

params_stack = {}
for model in exp['models']:
    model.model_name
    print(model.params)
    df = model.params[0]
    params_stack = []


[<model.Model object at 0x12fa4dd60>, <model.Model object at 0x12fa4ddc0>]
[{'p_size': 500, 'max_depth': 8, 'pc': 0.6, 'pm': 0.4, 'ngen': 50, 'verbose': False}]
[{'p_size': 250, 'max_depth': 8, 'pc': 0.6, 'pm': 0.4, 'ngen': 50, 'verbose': False}]


# Script analysis

In [ ]:


if 1:
    job = 'bagboost'
    experiment_name = 'bagboost_experiment'
    RH = ResultHandling(job, experiment_name, height=10, width = 10, box_label_size=30, target_dir='results_finished')
    
    for train in [True, False]:
        skipped_datasets = []
        latex_data = [] # at the end hsould be (ndatasets, nmodels, nseeds)
        training_string = 'train' if train else 'test'
        
        for ds in get_all_datasets():
            # Per model train
            plot_labels, plots = [], []
            for model_nam in RH.model_names:
                plots.append(RH.get_cond_data(mgen=False, train=train, model_name=model_nam, dataset_name=ds,col='full_acc'))
                plot_labels.append(model_nam)
            RH.box_plot(boxes=plots, labels=plot_labels, save=True, draw=False, figname=f'{job}_{ds}_{training_string}')
            if len(np.array(plots).shape) == 2:
                latex_data.append(np.array(plots))
            else:
                skipped_datasets.append(ds)
                
        # Create table 
        np_latex_data = np.array(latex_data)
        np_latex_data = np.mean(np_latex_data, axis=2)
        df = pd.DataFrame(data=np_latex_data, columns=RH.model_names)
        print(skipped_datasets)
        df.index = [ds for ds in get_all_datasets() if ds not in skipped_datasets]
        with open(f'tables/{job}_{training_string}.tex','w') as file:
            file.write(df.to_latex())



## Saving fast bag vs baseline gp vs basline bag

In [ ]:


if 1:
    job = 'fastbag_ham'
    experiment_name = 'fastbag'
    RH = ResultHandling(job, experiment_name, height=10, width = 10, box_label_size=30, target_dir='results_finished')
    other = ResultHandling('bagboost', 'bagboost_experiment', height=10, width = 10, box_label_size=30, target_dir='results_finished')
    for train in [True, False]:
        skipped_datasets = []
        latex_data = [] # at the end hsould be (ndatasets, nmodels, nseeds)
        training_string = 'train' if train else 'test'
        for ds in get_all_datasets():
            # Per model train
            plot_labels, plots = [], []
            for model_nam in other.model_names:
                if model_nam in ['GP', 'baggp']:
                    plots.append(other.get_cond_data(mgen=False, train=train, model_name=model_nam, dataset_name=ds,col='full_acc'))
                    plot_labels.append(model_nam)
            for model_nam in RH.model_names:
                plots.append(RH.get_cond_data(mgen=False, train=train, model_name=model_nam, dataset_name=ds,col='full_acc'))
                if model_nam == 'fastbaggp':
                    model_nam = 'rfbag'
                if model_nam == 'ham_fastbaggp':
                    model_nam = 'hrfbag'
                plot_labels.append(model_nam)
            RH.box_plot(boxes=plots, labels=plot_labels, save=True, draw=False, figname=f'{job}_{ds}_{training_string}')
            if len(np.array(plots).shape) == 2:
                latex_data.append(np.array(plots))
            else:
                skipped_datasets.append(ds)
                
        # Create table 
        np_latex_data = np.array(latex_data)
        np_latex_data = np.mean(np_latex_data, axis=2)
        df = pd.DataFrame(data=np_latex_data, columns=['gp', 'baggp', 'rfbag', 'rfham'])
        print(skipped_datasets)
        df.index = [ds for ds in get_all_datasets() if ds not in skipped_datasets]
        with open(f'tables/{job}_{training_string}.tex','w') as file:
            file.write(df.to_latex())
        
        
        



# PCA analysis


In [ ]:
if 1:
    job = 'pca'
    base = ResultHandling('bagboost', 'bagboost_experiment', height=10, width = 10, box_label_size=30, target_dir='results_finished')
    pca = ResultHandling('pca', 'dr', height=10, width = 10, box_label_size=30, target_dir='results_finished')
    
    for train in [True, False]:
        
        latex_data = [] # at the end hsould be (ndatasets, nmodels, nseeds)
        training_string = 'train' if train else 'test'
        skipped_datasets = []
        for ds in get_all_datasets():
            # Per model train
            plot_labels, plots = [], []
            for model_nam in base.model_names:
                if model_nam in ['GP', 'baggp']:
                    plots.append(base.get_cond_data(mgen=False, train=train, model_name=model_nam, dataset_name=ds,col='full_acc'))
                    plot_labels.append(model_nam)

            for i in range(4):
                j = i + 1
                plots.append(pca.get_cond_data(mgen=False, train=train, model_name='pcabag', dataset_name=f'{ds}_{j}',col='full_acc'))
                plot_labels.append(f'pca_{j}')
            pca.box_plot(boxes=plots, labels=plot_labels, save=True, draw=False, figname=f'{job}_{ds}_{training_string}')
            
            # 
            if len(np.array(plots).shape) == 2:
                latex_data.append(np.array(plots))
            else:
                skipped_datasets.append(ds)

        # Create table 
        np_latex_data = np.array(latex_data)
        np_latex_data = np.mean(np_latex_data, axis=2)
        df = pd.DataFrame(data=np_latex_data, columns=['gp', 'baggp', 'pca1', 'pca2', 'pca3', 'pca4'])
        df.index = [ds for ds in get_all_datasets() if ds not in skipped_datasets]
        with open(f'tables/{job}_{training_string}.tex','w') as file:
            file.write(df.to_latex())


# Analysis bag m3gp

In [ ]:
if 1:
    job = 'm3gpbag'
    base = ResultHandling('bagboost', 'bagboost_experiment', height=10, width = 10, box_label_size=30, target_dir='results_finished')
    m3gpbag = ResultHandling('m3gpbag', 'm3gp_bag', height=10, width = 10, box_label_size=30, target_dir='results_finished')
    
    for train in [True, False]:
        
        latex_data = [] # at the end hsould be (ndatasets, nmodels, nseeds)
        training_string = 'train' if train else 'test'
        skipped_datasets = []
        
        for ds in get_all_datasets():
            # Per model train
            plot_labels, plots = [], []
            for model_nam in base.model_names:
                if model_nam in ['GP', 'baggp']:
                    plots.append(base.get_cond_data(mgen=False, train=train, model_name=model_nam, dataset_name=ds,col='full_acc'))
                    plot_labels.append(model_nam)
            for model_nam in m3gpbag.model_names:
                plots.append(m3gpbag.get_cond_data(mgen=False, train=train, model_name=model_nam, dataset_name=ds,col='full_acc'))
                plot_labels.append(model_nam)
            m3gpbag.box_plot(boxes=plots, labels=plot_labels, save=True, draw=False, figname=f'{job}_{ds}_{training_string}')
            if len(np.array(plots).shape) == 2:
                latex_data.append(np.array(plots))
            else:
                skipped_datasets.append(ds)
                
        # Create table 
        np_latex_data = np.array(latex_data)
        np_latex_data = np.mean(np_latex_data, axis=2)
        df = pd.DataFrame(data=np_latex_data, columns=['GP', 'BagModel', 'm3gp'])
        df.index = [ds for ds in get_all_datasets() if ds not in skipped_datasets]
        with open(f'tables/{job}_{training_string}.tex','w') as file:
            file.write(df.to_latex())


# Analysis bag MOGP

In [ ]:
if 1:
    job = 'bagmogp'
    base = ResultHandling('bagboost', 'bagboost_experiment', height=10, width = 10, box_label_size=30, target_dir='results_finished')
    mogpbag = ResultHandling('bagmogp', 'bagmogp', height=10, width = 10, box_label_size=30, target_dir='results_finished')
    mogp500 = ResultHandling('mogp500', 'bagmogp', height=10, width = 10, box_label_size=30, target_dir='results_finished')
    
    for train in [True, False]:
        latex_data = [] # at the end hsould be (ndatasets, nmodels, nseeds)
        training_string = 'train' if train else 'test'
        skipped_datasets = []
        
        for ds in get_all_datasets():
            plot_labels, plots = [], []

            for model_nam in base.model_names:
                if model_nam in ['GP', 'baggp']:
                    plots.append(base.get_cond_data(mgen=False, train=train, model_name=model_nam, dataset_name=ds,col='full_acc'))
                    plot_labels.append(model_nam)

            for model_nam in mogpbag.model_names:
                plots.append(mogpbag.get_cond_data(mgen=False, train=train, model_name=model_nam, dataset_name=ds,col='full_acc'))
                plot_labels.append(model_nam)

            for model_nam in mogp500.model_names:
                plots.append(mogp500.get_cond_data(mgen=False, train=train, model_name=model_nam, dataset_name=ds,col='full_acc'))
                plot_labels.append(model_nam)
            mogpbag.box_plot(boxes=plots, labels=plot_labels, save=True, draw=False, figname=f'{job}_{ds}_{training_string}')

            if len(np.array(plots).shape) == 2:
                latex_data.append(np.array(plots))
            else:
                skipped_datasets.append(ds)
        
        # Create table 
        np_latex_data = np.array(latex_data)
        np_latex_data = np.mean(np_latex_data, axis=2)
        df = pd.DataFrame(data=np_latex_data, columns=['GP', 'BagModel', 'mogp250', 'mogp500'])
        df.index = [ds for ds in get_all_datasets() if ds not in skipped_datasets]
        with open(f'tables/{job}_{training_string}.tex','w') as file:
            file.write(df.to_latex())


# Script for generating table and wilson cox test

In [ ]:
f = RH.get_model_vs_dataset_dataframe(training=True)
print(f.to_latex())

In [ ]:
f = RH.get_model_vs_dataset_dataframe(training=False)
print(f.to_latex())

In [ ]:
RH.wilcoxon_test()

In [ ]:
q = np.array([1,2,3])
print(np.mean(q))
print(type(float(np.mean(q))))

# Fast GP

In [ ]:
RH.model_names

In [ ]:
# For a given experiment we use training and test per experiment 

RH = ResultHandling(job, experiment_name, height=10, width = 10, box_label_size=30, target_dir='results_finished')
f = RH.get_model_vs_dataset_dataframe(training=False)
print(type(f.to_latex()))
with open('sometexfile.tex','w') as file:
    file.write(f.to_latex())

In [ ]:
print(pd.DataFrame(data=np.array(plots).transpose()).to_latex())